# _Development: Nov. 12, 2019_

**_Today's Focus_**

To set up a simple `Streamlit` app that pulls my Tweets and shows 3 visualizations regarding the information. 

In [22]:
from joetools import private
import pandas as pd
import tweepy
import os
import time
import random
import sys
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [23]:
def environment_variables():
    # Set environment variables
    os.environ['TWITTER_API_KEY'] = private.TWITTER_API_KEY
    os.environ['TWITTER_API_SECRET'] = private.TWITTER_API_SECRET
    os.environ['TWITTER_TOKEN'] = private.TWITTER_TOKEN
    os.environ['TWITTER_TOKEN_SECRET'] = private.TWITTER_TOKEN_SECRET
    
def api_setup():
    '''set up Twitter API'''
    # run to assign environment variables 
    environment_variables()

    # access tweepy authenticator
    auth = tweepy.OAuthHandler(os.environ.get('TWITTER_API_KEY'), os.environ.get('TWITTER_API_SECRET'))
    auth.set_access_token(os.environ.get('TWITTER_TOKEN'), os.environ.get('TWITTER_TOKEN_SECRET'))

    # pass in above authentication to API
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    
    return api

def main():
    api = api_setup()
    # verify my credentials
    try:
        api.verify_credentials()
        print("Authentication verified.")
    except:
        print("Error during authentication.")
        
def sentiment_analyzer_scores(text):
    analyzer = SentimentIntensityAnalyzer()
    score = analyzer.polarity_scores(text)
    return score

def load_data(tweets_list):
    data = pd.DataFrame(tweets_list)
    return data

#@st.cache
def get_user_tweets(username):
    # instantiate empty list
    tweets_list = []
    
    for tweet in tweepy.Cursor(api.user_timeline, id=username, tweet_mode='extended').items(100):
        tweet_dict = {'created_at': tweet.created_at,
                     'tweet': tweet.full_text,
                     'positivity': sentiment_analyzer_scores(tweet.full_text)['pos'],
                     'negativity': sentiment_analyzer_scores(tweet.full_text)['neg']
                     }
        tweets_list.append(tweet_dict)
        
    # input created csv into pandas and return as DataFrame
    data = load_data(tweets_list)
    return data

In [24]:
environment_variables()

In [25]:
# access tweepy authenticator
auth = tweepy.OAuthHandler(os.environ.get('TWITTER_API_KEY'), os.environ.get('TWITTER_API_SECRET'))
auth.set_access_token(os.environ.get('TWITTER_TOKEN'), os.environ.get('TWITTER_TOKEN_SECRET'))

# pass in above authentication to API
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [26]:
# verify my credentials
try:
    api.verify_credentials()
    print("Authentication verified.")
except:
    print("Error during authentication.")

Authentication verified.


In [27]:
data = get_user_tweets('earny_joe')

In [28]:
data.head()

,created_at,negativity,positivity,tweet
0,2019-11-11 18:55:18,0.000,0.227,RT @TimothyKassis: We're excited to release a ...
1,2019-11-11 04:20:35,0.000,0.000,A few code examples for working with the Twitt...
2,2019-11-11 02:27:51,0.000,0.000,RT @DynamicWebPaige: ✨📒 @GoogleColab now has @...
3,2019-11-09 13:56:26,0.224,0.000,RT @peterpomeranzev: Information War has repla...
4,2019-11-09 05:33:23,0.000,0.415,"Day 10...to be honest, pleasantly surprised I ..."


In [29]:
data.tail()

,created_at,negativity,positivity,tweet
95,2019-10-17 18:38:59,0.0,0.000,RT @math_rachel: Come work with me as a Data E...
96,2019-10-17 16:09:35,0.0,0.000,RT @anthilemoon: “Originality is undetected pl...
97,2019-10-16 23:34:02,0.0,0.000,RT @srmsoumya: I will be speaking at O'Reilly ...
98,2019-10-16 17:32:36,0.0,0.299,RT @ansjin: I just published: How to create an...
99,2019-10-16 15:06:19,0.0,0.435,"RT @MLWhiz: Thanks, @ThiagoTDotCom for the sha..."
